In [1]:
using Base.Test
using BenchmarkTools

In [323]:
module nowrap

import Base: getindex, show
# import WeakRefStrings: WeakRefString
# import FunctionWrappers: FunctionWrapper

immutable Statistics
    word::String
end

immutable Feature
    f::Function
    description::String
end

size(::Feature) = ()
getindex(f::Feature, ::CartesianIndex{0}) = f

function allfeatures()
    features = Feature[]
    
    for l in 'a':'z'
        push!(features, Feature(stat -> l in stat.word, "contains '$l'"))
    end

    for l in 'a':'z'
        for j in 1:26
            push!(features, Feature(stat -> length(stat.word) >= j && stat.word[j] == l, "contains '$l' at index $j"))
        end
    end
    features
end

# macro feature(expr, description)
#     quote
#         push!(FEATURES, Feature($(esc(expr)), $description))
#     end
# end

show(io::IO, feature::Feature) = print(io, "Feature: `$(feature.description)`")

(f::Feature)(stat::Statistics)::Bool = f.f(stat)::Bool
satisfies(f::Feature, stat::Statistics)::Bool = f.f(stat)::Bool

end

nowrap

In [334]:
fs = nowrap.allfeatures()
f = fs[1]
s = "hello"
stat = nowrap.Statistics(s)

nowrap.Statistics("hello")

In [331]:
@benchmark $f($stat)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     91.999 ns (0.00% GC)
  median time:      97.143 ns (0.00% GC)
  mean time:        105.977 ns (0.00% GC)
  maximum time:     439.447 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     950
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [326]:
[nowrap.satisfies(f, stat) for f in fs]

702-element Array{Bool,1}:
 false
 false
 false
 false
  true
 false
 false
  true
 false
 false
 false
  true
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [327]:
@benchmark [g($stat) for g in $fs]

BenchmarkTools.Trial: 
  memory estimate:  848.00 bytes
  allocs estimate:  3
  --------------
  minimum time:     68.571 μs (0.00% GC)
  median time:      75.004 μs (0.00% GC)
  mean time:        83.222 μs (0.00% GC)
  maximum time:     375.349 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [328]:
function applyall(stat, fs)
    x = nowrap.satisfies(fs[1], stat)
end

applyall (generic function with 1 method)

In [329]:
@code_warntype applyall(stat, fs)

Variables:
  #self#::#applyall
  stat::nowrap.Statistics
  fs::Array{nowrap.Feature,1}
  x::Bool

Body:
  begin 
      SSAValue(1) = (Base.arrayref)(fs::Array{nowrap.Feature,1},1)::nowrap.Feature
      # meta: location  satisfies 0
      SSAValue(2) = (Core.typeassert)(((Core.getfield)(SSAValue(1),:f)::F)(stat::nowrap.Statistics)::ANY,nowrap.Bool)::Bool
      # meta: pop location
      SSAValue(0) = SSAValue(2)
      x::Bool = SSAValue(0)
      return SSAValue(0)
  end::Bool
